In [ ]:
# here only executting stuff. no functions or classes

In [ ]:
## um die models und functions hier nutzen zu können, muss man die datein als module importieren. 
## das geht auch wenn das notebeooks sind, ist aber deutlich einfacher, wenn es einfach .py dateien sind.

In [ ]:
!pip install -r requirements.txt

## CycleGAN Maps 


In [1]:
#!/usr/bin/env python3
#-*- coding:utf-8 -*-
#$ -l cuda=1 # remove this line when no GPU is needed!
#$ -q all.q # do not fill the qlogin queue
#$ -cwd # start processes in current working directory
#$ -V # provide environment variables to processes
#$ -t 1-8 # start 8 instances: to train different models in parallel
#Cluster settings, 

try:
    model_param_id = int(os.environ['SGE_TASK_ID'])
except:
    print("no SGE_TASK_ID set, choosing default model parameters ")
    model_param_id = 0 #param_train_cycle_list[0] should be default model params

import importlib
import models
import functions 
import torch
import torch.nn as nn
importlib.reload(models)
importlib.reload(functions)
import numpy as np
#different import way for cluster
'''
import imp
models = imp.load_source('models', './models.py')
functions = imp.load_source('functions', './functions.py')
'''

# init CycleGAN
genA2B = models.Generator(input_nc=3, output_nc=3, n_residual_blocks=9)
genB2A = models.Generator(input_nc=3, output_nc=3, n_residual_blocks=9)
discA  = models.Discriminator(input_nc=3)
discB  = models.Discriminator(input_nc=3)
classifier = models.Classifier().net
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
databaseName = "cityscapes"
root_path_data = "./data/"+databaseName
root_path_checkpoints = "./checkpoints/"+databaseName

targetEpoch = 2
saveAt = 1 #how often to store the model at training
print("the device ist at")
print(device)

if device == torch.device("cuda"):
    #debug
    print("should load models to cuda")
    print("device == cuda -> "+ str(device == "cuda:0"))
    #model.load_state_dict(torch.load("{}/{} epoch{}".format(path, name, epoch)))
else:
    #debug
    print("should load models to cpu")
    print("device == cuda -> "+ str(device == "cuda:0"))
    #model.load_state_dict(torch.load("{}/{} epoch{}".format(path, name, epoch),map_location=torch.device('cpu')))

cycle  = models.CycleGAN(genA2B, genB2A, discA, discB, classifier, device, root_path_data, root_path_checkpoints)


param_train1  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r9_advMSE_l10", resnet_blocks=9, loss_adv=torch.nn.MSELoss(), lambdas=(10,0.5)) # default
param_train2  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r9_advMSE_l5" , resnet_blocks=9, loss_adv=torch.nn.MSELoss(), lambdas=(5,0.5))
param_train3  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r9_advL1_l10", resnet_blocks=9, loss_adv=torch.nn.L1Loss(), lambdas=(10,0.5))
param_train4  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r9_advL1_l5" , resnet_blocks=9, loss_adv=torch.nn.L1Loss(), lambdas=(5,0.5))
param_train5  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r5_advMSE_l10", resnet_blocks=5, loss_adv=torch.nn.MSELoss(), lambdas=(10,0.5))
param_train6  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r5_advMSE_l5" , resnet_blocks=5, loss_adv=torch.nn.MSELoss(), lambdas=(5,0.5))
param_train7  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r5_advL1_l10", resnet_blocks=5, loss_adv=torch.nn.L1Loss(), lambdas=(10,0.5))
param_train8  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r5_advL1_l5" , resnet_blocks=5, loss_adv=torch.nn.L1Loss(), lambdas=(5,0.5))
param_train_cycle_list = [param_train1, param_train2, param_train3, param_train4, param_train5, param_train6, param_train7, param_train8]

# paramter to load the background error verison. since this extension needed major changes in training/loaded training was done locally.
param_back_err = models.Param(name ="background_default", epochs = 80)

# paramters needed to call eval_testset() function
param_eval_testset = models.Param(channels = 3, size= 256) 

# parameters used for training the classifier
param_train_classifier = models.Param(channels = 3, epochs = targetEpoch, size = 256, name = "classifier_test")

no SGE_TASK_ID set, choosing default model parameters 
the device ist at
cuda
should load models to cuda
device == cuda -> False


## Train Model

In [2]:
param_train_cycle = param_train_cycle_list[model_param_id-1]
cycle.train(param_train_cycle)   

training started!...
logfile running at ./checkpoints/cityscapes/logcycle_r5_advL1_l5.txt
debug: converted to rgb!
batch 1 done in 0.4699 seconds, cycle_loss:1.0568004846572876
debug: converted to rgb!
batch 2 done in 0.3973 seconds, cycle_loss:0.87577223777771
debug: converted to rgb!
batch 3 done in 0.3979 seconds, cycle_loss:0.9412569999694824
debug: converted to rgb!
batch 4 done in 0.3917 seconds, cycle_loss:0.7922477722167969
debug: converted to rgb!
batch 5 done in 0.3921 seconds, cycle_loss:0.8007247447967529
debug: converted to rgb!
batch 6 done in 0.3917 seconds, cycle_loss:0.7617536783218384
debug: converted to rgb!
batch 7 done in 0.3988 seconds, cycle_loss:0.6319853067398071
debug: converted to rgb!
batch 8 done in 0.3992 seconds, cycle_loss:0.6981194019317627
debug: converted to rgb!
batch 9 done in 0.3999 seconds, cycle_loss:0.6247454881668091
debug: converted to rgb!
batch 10 done in 0.4332 seconds, cycle_loss:0.6556342244148254
debug: converted to rgb!
batch 11 done in

batch 93 done in 0.3997 seconds, cycle_loss:0.3795819878578186
debug: converted to rgb!
batch 94 done in 0.4049 seconds, cycle_loss:0.3712737560272217
debug: converted to rgb!
batch 95 done in 0.4201 seconds, cycle_loss:0.31437236070632935
debug: converted to rgb!
batch 96 done in 0.4229 seconds, cycle_loss:0.5953971147537231
debug: converted to rgb!
batch 97 done in 0.4213 seconds, cycle_loss:0.46614885330200195
debug: converted to rgb!
batch 98 done in 0.4001 seconds, cycle_loss:0.3962574601173401
debug: converted to rgb!
batch 99 done in 0.4178 seconds, cycle_loss:0.36992591619491577
debug: converted to rgb!
batch 100 done in 0.4226 seconds, cycle_loss:0.3937932550907135
debug: converted to rgb!
batch 101 done in 0.3997 seconds, cycle_loss:0.36676275730133057
debug: converted to rgb!
batch 102 done in 0.4203 seconds, cycle_loss:0.2617760896682739
debug: converted to rgb!
batch 103 done in 0.4078 seconds, cycle_loss:0.34615904092788696
debug: converted to rgb!
batch 104 done in 0.414

batch 185 done in 0.401 seconds, cycle_loss:0.2950386106967926
debug: converted to rgb!
batch 186 done in 0.3961 seconds, cycle_loss:0.29235726594924927
debug: converted to rgb!
batch 187 done in 0.3951 seconds, cycle_loss:0.30360594391822815
debug: converted to rgb!
batch 188 done in 0.401 seconds, cycle_loss:0.3332362771034241
debug: converted to rgb!
batch 189 done in 0.4003 seconds, cycle_loss:0.3478062152862549
debug: converted to rgb!
batch 190 done in 0.3964 seconds, cycle_loss:0.3725743889808655
debug: converted to rgb!
batch 191 done in 0.3929 seconds, cycle_loss:0.2835233509540558
debug: converted to rgb!
batch 192 done in 0.3919 seconds, cycle_loss:0.29582950472831726
debug: converted to rgb!
batch 193 done in 0.3961 seconds, cycle_loss:0.297945499420166
debug: converted to rgb!
batch 194 done in 0.4197 seconds, cycle_loss:0.30078887939453125
debug: converted to rgb!
batch 195 done in 0.4053 seconds, cycle_loss:0.3359166383743286
debug: converted to rgb!
batch 196 done in 0.

batch 277 done in 0.3974 seconds, cycle_loss:0.27649110555648804
debug: converted to rgb!
batch 278 done in 0.4182 seconds, cycle_loss:0.28747767210006714
debug: converted to rgb!
batch 279 done in 0.3961 seconds, cycle_loss:0.4182385802268982
debug: converted to rgb!
batch 280 done in 0.3933 seconds, cycle_loss:0.512552797794342
debug: converted to rgb!
batch 281 done in 0.3993 seconds, cycle_loss:0.26301833987236023
debug: converted to rgb!
batch 282 done in 0.4243 seconds, cycle_loss:0.2948460578918457
debug: converted to rgb!
batch 283 done in 0.4059 seconds, cycle_loss:0.3423653841018677
debug: converted to rgb!
batch 284 done in 0.4015 seconds, cycle_loss:0.335712730884552
debug: converted to rgb!
batch 285 done in 0.4254 seconds, cycle_loss:0.36493632197380066
debug: converted to rgb!
batch 286 done in 0.3945 seconds, cycle_loss:0.26849132776260376
debug: converted to rgb!
batch 287 done in 0.416 seconds, cycle_loss:0.3061348497867584
debug: converted to rgb!
batch 288 done in 0

batch 369 done in 0.4038 seconds, cycle_loss:0.49920302629470825
debug: converted to rgb!
batch 370 done in 0.3987 seconds, cycle_loss:0.33001893758773804
debug: converted to rgb!
batch 371 done in 0.4035 seconds, cycle_loss:0.29502803087234497
debug: converted to rgb!
batch 372 done in 0.3999 seconds, cycle_loss:0.40049824118614197
debug: converted to rgb!
batch 373 done in 0.3992 seconds, cycle_loss:0.4273015558719635
debug: converted to rgb!
batch 374 done in 0.3998 seconds, cycle_loss:0.2515067160129547
debug: converted to rgb!
batch 375 done in 0.3982 seconds, cycle_loss:0.2817670702934265
debug: converted to rgb!
batch 376 done in 0.4032 seconds, cycle_loss:0.3328847885131836
debug: converted to rgb!
batch 377 done in 0.4006 seconds, cycle_loss:0.34764713048934937
debug: converted to rgb!
batch 378 done in 0.4011 seconds, cycle_loss:0.3607305884361267
debug: converted to rgb!
batch 379 done in 0.3991 seconds, cycle_loss:0.22225920855998993
debug: converted to rgb!
batch 380 done 

...

batch 2600 done in 0.4178 seconds, cycle_loss:0.19372490048408508
debug: converted to rgb!
batch 2601 done in 0.4193 seconds, cycle_loss:0.6260506510734558
debug: converted to rgb!
batch 2602 done in 0.4111 seconds, cycle_loss:0.2717166543006897
debug: converted to rgb!
batch 2603 done in 0.4086 seconds, cycle_loss:0.22426703572273254
debug: converted to rgb!
batch 2604 done in 0.4112 seconds, cycle_loss:0.27365338802337646
debug: converted to rgb!
batch 2605 done in 0.413 seconds, cycle_loss:0.17756684124469757
debug: converted to rgb!
batch 2606 done in 0.4044 seconds, cycle_loss:0.4876902997493744
debug: converted to rgb!
batch 2607 done in 0.415 seconds, cycle_loss:0.3200414478778839
debug: converted to rgb!
batch 2608 done in 0.4136 seconds, cycle_loss:0.22687840461730957
debug: converted to rgb!
batch 2609 done in 0.4075 seconds, cycle_loss:0.21169957518577576
debug: converted to rgb!
batch 2610 done in 0.4199 seconds, cycle_loss:0.25514134764671326
debug: converted to rgb!
batch

batch 2691 done in 0.4131 seconds, cycle_loss:0.2054964303970337
debug: converted to rgb!
batch 2692 done in 0.4033 seconds, cycle_loss:0.14195556938648224
debug: converted to rgb!
batch 2693 done in 0.42 seconds, cycle_loss:0.4110727310180664
debug: converted to rgb!
batch 2694 done in 0.4112 seconds, cycle_loss:0.17130890488624573
debug: converted to rgb!
batch 2695 done in 0.4142 seconds, cycle_loss:0.19234484434127808
debug: converted to rgb!
batch 2696 done in 0.4057 seconds, cycle_loss:0.13790862262248993
debug: converted to rgb!
batch 2697 done in 0.4064 seconds, cycle_loss:0.19759836792945862
debug: converted to rgb!
batch 2698 done in 0.4147 seconds, cycle_loss:0.27499011158943176
debug: converted to rgb!
batch 2699 done in 0.4207 seconds, cycle_loss:0.30643558502197266
debug: converted to rgb!
batch 2700 done in 0.408 seconds, cycle_loss:0.17280682921409607
debug: converted to rgb!
batch 2701 done in 0.3946 seconds, cycle_loss:0.18802523612976074
debug: converted to rgb!
batc

batch 2782 done in 0.3781 seconds, cycle_loss:0.17481866478919983
debug: converted to rgb!
batch 2783 done in 0.39 seconds, cycle_loss:0.2853914797306061
debug: converted to rgb!
batch 2784 done in 0.3767 seconds, cycle_loss:0.15187181532382965
debug: converted to rgb!
batch 2785 done in 0.3751 seconds, cycle_loss:0.24774599075317383
debug: converted to rgb!
batch 2786 done in 0.3763 seconds, cycle_loss:0.20757810771465302
debug: converted to rgb!
batch 2787 done in 0.376 seconds, cycle_loss:0.21155542135238647
debug: converted to rgb!
batch 2788 done in 0.378 seconds, cycle_loss:0.21413496136665344
debug: converted to rgb!
batch 2789 done in 0.4084 seconds, cycle_loss:0.1802978217601776
debug: converted to rgb!
batch 2790 done in 0.3839 seconds, cycle_loss:0.20209071040153503
debug: converted to rgb!
batch 2791 done in 0.3772 seconds, cycle_loss:0.16940802335739136
debug: converted to rgb!
batch 2792 done in 0.3803 seconds, cycle_loss:0.19662903249263763
debug: converted to rgb!
batch

batch 2873 done in 0.3781 seconds, cycle_loss:0.19413159787654877
debug: converted to rgb!
batch 2874 done in 0.3787 seconds, cycle_loss:0.28582966327667236
debug: converted to rgb!
batch 2875 done in 0.376 seconds, cycle_loss:0.22033993899822235
debug: converted to rgb!
batch 2876 done in 0.382 seconds, cycle_loss:0.21167710423469543
debug: converted to rgb!
batch 2877 done in 0.3839 seconds, cycle_loss:0.22026526927947998
debug: converted to rgb!
batch 2878 done in 0.3795 seconds, cycle_loss:0.15028078854084015
debug: converted to rgb!
batch 2879 done in 0.3869 seconds, cycle_loss:0.2056058645248413
debug: converted to rgb!
batch 2880 done in 0.3809 seconds, cycle_loss:0.1480656862258911
debug: converted to rgb!
batch 2881 done in 0.3786 seconds, cycle_loss:0.2142641246318817
debug: converted to rgb!
batch 2882 done in 0.3792 seconds, cycle_loss:0.24094703793525696
debug: converted to rgb!
batch 2883 done in 0.3843 seconds, cycle_loss:0.19003629684448242
debug: converted to rgb!
batc

batch 2964 done in 0.4053 seconds, cycle_loss:0.3046211898326874
debug: converted to rgb!
batch 2965 done in 0.4055 seconds, cycle_loss:0.248378723859787
debug: converted to rgb!
batch 2966 done in 0.4073 seconds, cycle_loss:0.27275699377059937
debug: converted to rgb!
batch 2967 done in 0.4069 seconds, cycle_loss:0.16811929643154144
debug: converted to rgb!
batch 2968 done in 0.4046 seconds, cycle_loss:0.22718271613121033
debug: converted to rgb!
batch 2969 done in 0.4085 seconds, cycle_loss:0.22147303819656372
debug: converted to rgb!
batch 2970 done in 0.4085 seconds, cycle_loss:0.1510453224182129
debug: converted to rgb!
batch 2971 done in 0.4089 seconds, cycle_loss:0.14811497926712036
debug: converted to rgb!
batch 2972 done in 0.4081 seconds, cycle_loss:0.18582919239997864
debug: converted to rgb!
batch 2973 done in 0.4078 seconds, cycle_loss:0.16569027304649353
debug: converted to rgb!
batch 2974 done in 0.4086 seconds, cycle_loss:0.1560898721218109
debug: converted to rgb!
batc

## Generate Results

In [2]:
#for every model --TODO
for paramItem in param_train_cycle_list:
    #load model
    cycle.load_cycle_nets(epoch = 100, model_name = paramItem.name)
    print(device)
    #generate for every test image some output image
    cycle._create_evalset_paired(paramItem,subfolder=paramItem.name)
    #calculate error with expected image

#compare errors between models

#run pix2pix ? and compare it with that?


loading models to cuda
loading models to cuda
loading models to cuda
loading models to cuda
cuda
creating save folder: ./data/maps/evalA/cycle_r9_advMSE_l10  and  ./data/maps/evalB/cycle_r9_advMSE_l10
finished domain A
finished domain B
loading models to cuda
loading models to cuda
loading models to cuda
loading models to cuda
cuda
creating save folder: ./data/maps/evalA/cycle_r9_advMSE_l5  and  ./data/maps/evalB/cycle_r9_advMSE_l5
finished domain A
finished domain B
loading models to cuda
An error occured while loading the model:
./checkpoints/maps/netG_A2Bcycle_r9_advL1_l10 epoch100
[Errno 2] No such file or directory: './checkpoints/maps/netG_A2Bcycle_r9_advL1_l10 epoch100'
loading models to cuda
An error occured while loading the model:
./checkpoints/maps/netG_B2Acycle_r9_advL1_l10 epoch100
[Errno 2] No such file or directory: './checkpoints/maps/netG_B2Acycle_r9_advL1_l10 epoch100'
loading models to cuda
An error occured while loading the model:
./checkpoints/maps/netD_Acycle_r9_a

## Calculate Error for each model

In [3]:
#MSE? SSIM? FCN? 
import os
from PIL import Image
import torchvision.transforms as transforms


losses= {}
for paramItem in param_train_cycle_list:
    source_domains = ["A", "B"]
    subfolder = paramItem.name
    losses[paramItem.name]={}
    for source in source_domains:  
        losses[paramItem.name][source]=[]
        folder=root_path_data + "/eval{}/{}/".format(source,subfolder)
        folderSize = int(len(os.listdir(folder)) / 3)
        for i in range(0,folderSize):
            gen="{}{}_generated.jpg".format(folder, i)
            exp="{}{}_expected.jpg".format(folder, i)
            gen=Image.open(gen)
            exp=Image.open(exp)
            loss=functions.calcL1(gen,exp)
            losses[paramItem.name][source].append(loss)
    

## display losses

In [4]:
for paramItem in param_train_cycle_list:
    source_domains = ["A", "B"]
    for source in source_domains:
        summ = np.array(losses[paramItem.name][source]).sum()
        std = np.std(losses[paramItem.name][source])
        overall = np.array(losses[paramItem.name]['A']).sum() + np.array(losses[paramItem.name]['B']).sum()
        print("param:{} source:{} std:{:2f} sum:{:0f} overall:{}".format(paramItem.name,source,std,summ,overall))

param:cycle_r9_advMSE_l10 source:A std:0.020727 sum:42.331000 overall:200.31400000000002
param:cycle_r9_advMSE_l10 source:B std:0.027450 sum:157.983000 overall:200.31400000000002
param:cycle_r9_advMSE_l5 source:A std:0.019033 sum:41.606000 overall:198.77599999999998
param:cycle_r9_advMSE_l5 source:B std:0.027498 sum:157.170000 overall:198.77599999999998
param:cycle_r9_advL1_l10 source:A std:0.019033 sum:41.606000 overall:198.77599999999998
param:cycle_r9_advL1_l10 source:B std:0.027498 sum:157.170000 overall:198.77599999999998
param:cycle_r9_advL1_l5 source:A std:0.018212 sum:40.860000 overall:205.512
param:cycle_r9_advL1_l5 source:B std:0.027137 sum:164.652000 overall:205.512
param:cycle_r5_advMSE_l10 source:A std:0.017783 sum:41.123000 overall:198.801
param:cycle_r5_advMSE_l10 source:B std:0.028439 sum:157.678000 overall:198.801
param:cycle_r5_advMSE_l5 source:A std:0.018145 sum:41.852000 overall:196.74
param:cycle_r5_advMSE_l5 source:B std:0.025475 sum:154.888000 overall:196.74
para

In [73]:
for paramItem in param_train_cycle_list:
    source_domains = ["A", "B"]
    for source in source_domains:
        summ = np.array(losses[paramItem.name][source]).sum()
        std = np.std(losses[paramItem.name][source])
        overall = np.array(losses[paramItem.name]['A']).sum() + np.array(losses[paramItem.name]['B']).sum()
        print("param:{} source:{} std:{:2f} sum:{:0f} overall:{}".format(paramItem.name,source,std,summ,overall))

param:cycle_r9_advMSE_l10 source:A std:0.021128 sum:44.468000 overall:207.642
param:cycle_r9_advMSE_l10 source:B std:0.027676 sum:163.174000 overall:207.642
param:cycle_r9_advMSE_l5 source:A std:0.018349 sum:41.879000 overall:212.426
param:cycle_r9_advMSE_l5 source:B std:0.027998 sum:170.547000 overall:212.426
param:cycle_r9_advL1_l10 source:A std:0.020923 sum:43.669000 overall:201.413
param:cycle_r9_advL1_l10 source:B std:0.026593 sum:157.744000 overall:201.413
param:cycle_r9_advL1_l5 source:A std:0.017752 sum:45.098000 overall:202.21200000000002
param:cycle_r9_advL1_l5 source:B std:0.026540 sum:157.114000 overall:202.21200000000002
param:cycle_r5_advMSE_l10 source:A std:0.023278 sum:45.731000 overall:211.471
param:cycle_r5_advMSE_l10 source:B std:0.028075 sum:165.740000 overall:211.471
param:cycle_r5_advMSE_l5 source:A std:0.021359 sum:44.727000 overall:205.86300000000003
param:cycle_r5_advMSE_l5 source:B std:0.027667 sum:161.136000 overall:205.86300000000003
param:cycle_r5_advL1_l10

In [ ]:
bla
for paramItem in param_train_cycle_list:
    print(type(paramItem.name))
    print(type("hallo"))

In [36]:
#calculate differences between 50 and 100
losses50 = np.load(root_path_data+"/pairedlosses50.npy",allow_pickle=True).item()
losses100 = np.load(root_path_data+"/pairedlosses100.npy",allow_pickle=True).item()
for paramItem in param_train_cycle_list:
    source_domains = ["A", "B"]
    for source in source_domains:
        summ50 = np.array(losses50[paramItem.name][source]).sum()
        std50 = np.std(losses50[paramItem.name][source])
        overall50 = np.array(losses50[paramItem.name]['A']).sum() + np.array(losses50[paramItem.name]['B']).sum()
        summ100 = np.array(losses100[paramItem.name][source]).sum()
        std100 = np.std(losses100[paramItem.name][source])
        overall100 = np.array(losses100[paramItem.name]['A']).sum() + np.array(losses100[paramItem.name]['B']).sum()
        diffstd = std100-std50
        diffsumm= summ100-summ50
        diffoverall= overall100-overall50
        print("param:{}\n source:{} \n std:{:2f}({:2f}) \n sum:{:0f}({:0f}) \n overall:{}({:2f})".format(paramItem.name,source,std100,diffstd,summ100,diffsumm,overall100,diffoverall))

param:cycle_r9_advMSE_l10
 source:A 
 std:0.020727(-0.000401) 
 sum:42.331000(-2.137000) 
 overall:200.31400000000002(-7.328000)
param:cycle_r9_advMSE_l10
 source:B 
 std:0.027450(-0.000226) 
 sum:157.983000(-5.191000) 
 overall:200.31400000000002(-7.328000)
param:cycle_r9_advMSE_l5
 source:A 
 std:0.019033(0.000683) 
 sum:41.606000(-0.273000) 
 overall:198.77599999999998(-13.650000)
param:cycle_r9_advMSE_l5
 source:B 
 std:0.027498(-0.000500) 
 sum:157.170000(-13.377000) 
 overall:198.77599999999998(-13.650000)
param:cycle_r9_advL1_l10
 source:A 
 std:0.019033(-0.001891) 
 sum:41.606000(-2.063000) 
 overall:198.77599999999998(-2.637000)
param:cycle_r9_advL1_l10
 source:B 
 std:0.027498(0.000904) 
 sum:157.170000(-0.574000) 
 overall:198.77599999999998(-2.637000)
param:cycle_r9_advL1_l5
 source:A 
 std:0.018212(0.000460) 
 sum:40.860000(-4.238000) 
 overall:205.512(3.300000)
param:cycle_r9_advL1_l5
 source:B 
 std:0.027137(0.000597) 
 sum:164.652000(7.538000) 
 overall:205.512(3.300000

In [29]:
losses50.item()

{'cycle_r9_advMSE_l10': {'A': [0.061,
   0.03,
   0.029,
   0.043,
   0.026,
   0.045,
   0.038,
   0.025,
   0.04,
   0.035,
   0.024,
   0.023,
   0.034,
   0.022,
   0.031,
   0.028,
   0.03,
   0.026,
   0.026,
   0.046,
   0.026,
   0.026,
   0.045,
   0.066,
   0.032,
   0.04,
   0.024,
   0.075,
   0.029,
   0.032,
   0.047,
   0.027,
   0.04,
   0.029,
   0.05,
   0.064,
   0.028,
   0.049,
   0.036,
   0.041,
   0.021,
   0.028,
   0.04,
   0.029,
   0.021,
   0.041,
   0.079,
   0.057,
   0.105,
   0.043,
   0.033,
   0.025,
   0.022,
   0.028,
   0.03,
   0.037,
   0.043,
   0.03,
   0.031,
   0.07,
   0.035,
   0.099,
   0.027,
   0.025,
   0.036,
   0.056,
   0.031,
   0.027,
   0.053,
   0.027,
   0.03,
   0.03,
   0.033,
   0.036,
   0.038,
   0.05,
   0.028,
   0.022,
   0.037,
   0.042,
   0.093,
   0.041,
   0.026,
   0.061,
   0.032,
   0.033,
   0.067,
   0.028,
   0.024,
   0.027,
   0.084,
   0.037,
   0.031,
   0.026,
   0.03,
   0.086,
   0.035,
   0.136,
   0.0

In [19]:
type(losses)

dict

In [40]:
model_param_id

0